In [2]:
import os

# global settings
base_path = '/mnt/data_lab513/vqtran_data'
root_data = os.path.join(base_path, "data", "raw_data", "ADNI_NIfTI")
root_bias_correction = os.path.join(base_path, "data", "clean_data", "mri_bias_correction")
root_bet = os.path.join(base_path, "data", "clean_data", "mri_brain_extraction")
root_reg = os.path.join(base_path, "data", "clean_data", "mri_registration")
root_meta = os.path.join(base_path, "data", "meta_data")#, "Pre-Thesis_metadata", "ADNI") 
root_train = os.path.join(base_path, "data", "train_data")
root_train_dec = os.path.join(base_path, "data", "data_train_dec", "origin")

In [3]:
len(os.listdir(root_data))


2091

In [4]:
len(os.listdir(root_train_dec))
# os.listdir(root_train_dec)[:5]

2090

In [5]:
os.listdir(root_train_dec)[:5]

['ADNI_002_S_0295_MR_MP-RAGE__br_raw_20060418193713091_1_S13408_I13722.nii.gz',
 'ADNI_002_S_0295_MR_MPRAGE_br_raw_20110602125255238_39_S110476_I238627.nii.gz',
 'ADNI_002_S_0413_MR_MP-RAGE__br_raw_20060502142705523_1_S13893_I14437.nii.gz',
 'ADNI_002_S_0413_MR_MPRAGE_br_raw_20110617102624650_40_S111992_I240812.nii.gz',
 'ADNI_002_S_0559_MR_MP-RAGE__br_raw_20060524075551726_1_S14875_I15948.nii.gz']

In [6]:
import pandas as pd


csv_table_dir = os.path.join(root_meta, "Pre-Thesis_5_21_2021_ADNI.csv")
df = pd.read_csv(csv_table_dir)

df.head(10)
# df.tail(30)

#print(df.to_string()) 

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I325119,941_S_4764,EMCI,F,83,24,MRI,MPRAGE,Original,8/20/2012,DCM,NaN
1,I307653,941_S_4764,EMCI,F,83,22,MRI,MPRAGE,Original,6/01/2012,DCM,11/22/2020
2,I294080,941_S_4420,EMCI,M,81,22,MRI,MPRAGE,Original,3/28/2012,DCM,11/22/2020
3,I313224,941_S_4420,EMCI,M,82,24,MRI,MPRAGE,Original,6/27/2012,DCM,NaN
4,I352818,941_S_4420,EMCI,M,82,25,MRI,MPRAGE,Original,12/21/2012,DCM,NaN
5,I296058,941_S_4377,LMCI,F,70,24,MRI,MPRAGE,Original,4/05/2012,DCM,NaN
6,I275228,941_S_4377,LMCI,F,69,22,MRI,MPRAGE,Original,1/04/2012,DCM,11/22/2020
7,I324376,941_S_4377,LMCI,F,70,25,MRI,MPRAGE,Original,8/16/2012,DCM,NaN
8,I269043,941_S_4376,CN,F,76,22,MRI,MPRAGE,Original,6/01/2011,DCM,3/11/2021
9,I276860,941_S_4376,CN,F,77,22,MRI,MPRAGE,Original,1/06/2012,DCM,2/25/2020


In [7]:
len(df)

2090

In [8]:
df.keys()

Index(['Image Data ID', 'Subject', 'Group', 'Sex', 'Age', 'Visit', 'Modality',
       'Description', 'Type', 'Acq Date', 'Format', 'Downloaded'],
      dtype='object')

In [9]:
num_unique_image = len(df["Image Data ID"].unique())
num_unique_subject = len(df["Subject"].unique())
print("Image per subject: {} / {} = {}".format(num_unique_image, num_unique_subject, float(num_unique_image)/float(num_unique_subject)))
#len(df["Group"].unique())

Image per subject: 2090 / 1004 = 2.0816733067729083


In [10]:
#get label from id_image
#Requirement: prevent data_leakage with id_subject
check_leakage_set = set()
check_leakage_list = list()
check_leakage_num = 0

for dirname, _, filenames in os.walk(root_train_dec):
    for filename in filenames:     
        # path = os.path.join(dirname, filename)
        # print(path)
        
        id_image= filename[filename.index('_I')+1 : filename.index('.nii')]
        # print(id_image)        
                
        id_subject = filename[5:15]
        # print(id_subject)
            
        if(id_subject in check_leakage_set):
            continue
        
        row_index = df.index[df['Image Data ID'] == id_image].tolist()
        label = df['Group'][row_index[0]]
        
        # print(label)
        check_leakage_set.add(id_subject)
        check_leakage_list.append(id_subject)
        check_leakage_num += 1
        #print("----------------")
        
        # break
    # break
        
print(check_leakage_num)
print(len(check_leakage_set))
print(len(check_leakage_list))
        
        #https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

1004
1004
1004


SEARCH LABEL BY ID_IMAGE OR ID_SUBJECT

In [40]:
# SEARCH_METRICS = "ID_SUBJECT" # "ID_IMAGE"
# ID_IMAGE = "I218391"
# ID_SUBJECT = "037_S_4001" # "037_S_4001" #

# SEARCH_METRICS = "ID_IMAGE"
# ID_IMAGE = "I41449"
# ID_SUBJECT = "094_S_1241"

SEARCH_METRICS = "ID_SUBJECT" # "ID_IMAGE"
ID_IMAGE = ""
ID_SUBJECT = "041_S_4271" # "037_S_4001" #


SUBJECT_DF_INDEX = 0

for dirname, _, filenames in os.walk(root_train_dec):
    for filename in filenames:
        if (SEARCH_METRICS == "ID_IMAGE"):          
            if (ID_IMAGE == filename[filename.index('_I')+1 : filename.index('.nii')]):
                print("FILENAME:",filename)
                
                id_image= filename[filename.index('_I')+1 : filename.index('.nii')]
                print("ID_IMAGE:",id_image)        
                        
                id_subject = filename[5:15]
                print("ID_SUBJECT:",id_subject)
                
                row_index = df.index[df['Image Data ID'] == id_image].tolist()
                SUBJECT_DF_INDEX = row_index[0]
                print("SUBJECT_DF_INDEX ", SUBJECT_DF_INDEX)
                label = df['Group'][row_index[0]]
                print("LABEL:",label)
                
        elif (SEARCH_METRICS == "ID_SUBJECT"):  
            if (ID_SUBJECT == filename[5:15]):
                print("FILENAME:",filename)
                
                id_image= filename[filename.index('_I')+1 : filename.index('.nii')]
                print("ID_IMAGE:",id_image)        
                        
                id_subject = filename[5:15]
                print("ID_SUBJECT:",id_subject)
                
                row_index = df.index[df['Image Data ID'] == id_image].tolist()
                SUBJECT_DF_INDEX = row_index[0]
                print("SUBJECT_DF_INDEX ", SUBJECT_DF_INDEX)
                print(row_index)
                label = df['Group'][row_index[0]]
                print("LABEL:",label)
        else:
            raise Exception("only 2 argument availabel: ID_IMAGE or ID_SUBJECT")

FILENAME: ADNI_041_S_4271_MR_MPRAGE_br_raw_20111011124558005_3_S125033_I260502.nii.gz
ID_IMAGE: I260502
ID_SUBJECT: 041_S_4271
SUBJECT_DF_INDEX  1086
[1086]
LABEL: EMCI
FILENAME: ADNI_041_S_4271_MR_MPRAGE_br_raw_20111011124607848_69_S125037_I260507.nii.gz
ID_IMAGE: I260507
ID_SUBJECT: 041_S_4271
SUBJECT_DF_INDEX  1084
[1084]
LABEL: EMCI
FILENAME: ADNI_041_S_4271_MR_MPRAGE_br_raw_20120124134836730_105_S138269_I280272.nii.gz
ID_IMAGE: I280272
ID_SUBJECT: 041_S_4271
SUBJECT_DF_INDEX  1087
[1087]
LABEL: EMCI
FILENAME: ADNI_041_S_4271_MR_MPRAGE_br_raw_20120426145658630_93_S148400_I300650.nii.gz
ID_IMAGE: I300650
ID_SUBJECT: 041_S_4271
SUBJECT_DF_INDEX  1085
[1085]
LABEL: EMCI
FILENAME: ADNI_041_S_4271_MR_MPRAGE_br_raw_20120426145701020_116_S148403_I300653.nii.gz
ID_IMAGE: I300653
ID_SUBJECT: 041_S_4271
SUBJECT_DF_INDEX  1088
[1088]
LABEL: EMCI


In [43]:
# print(SUBJECT_DF_INDEX)
SUBJECT_DF_INDEX = [1084, 1085 ,1086, 1087, 1088]
missing_image_row = df.loc[SUBJECT_DF_INDEX,:]
missing_image_row
# missing_image_row.to_frame().T

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
1084,I260507,041_S_4271,EMCI,M,62,22,MRI,MPRAGE,Original,10/07/2011,DCM,11/22/2020
1085,I300650,041_S_4271,EMCI,M,62,25,MRI,MPRAGE,Original,4/25/2012,DCM,NaN
1086,I260502,041_S_4271,EMCI,M,62,22,MRI,MPRAGE,Original,10/07/2011,DCM,11/22/2020
1087,I280272,041_S_4271,EMCI,M,62,24,MRI,MPRAGE,Original,1/23/2012,DCM,NaN
1088,I300653,041_S_4271,EMCI,M,62,25,MRI,MPRAGE,Original,4/25/2012,DCM,NaN


SEARCH WHAT ID_SUBJECT ARE NOT AVAILABLE 

GET ALL SUBJECT NAME FROM OS DIR

In [13]:
os_id_subject_list = []

for dirname, _, filenames in os.walk(root_train_dec):
    for filename in filenames:                 
        # print("FILENAME:",filename)
            
        # id_image= filename[filename.index('_I')+1 : filename.index('.nii')]
        # print("ID_IMAGE:",id_image)        
                    
        id_subject = filename[5:15]
        # print("ID_SUBJECT:",id_subject)
            
        # row_index = df.index[df['Image Data ID'] == id_image].tolist()
        # label = df['Group'][row_index[0]]
        # print("LABEL:",label)
        
        os_id_subject_list.append(id_subject)
        
print(os_id_subject_list[:5])       
print(len(os_id_subject_list))   

['002_S_0295', '002_S_0295', '002_S_0413', '002_S_0413', '002_S_0559']
2090


GET ALL SUBJECT NAME FROM DATAFRAME

In [14]:
df_id_subject_list = []
df_id_subject_list = df["Subject"].to_list()
      
print(len(df_id_subject_list))  
print(df_id_subject_list[:5]) 


2090
['941_S_4764', '941_S_4764', '941_S_4420', '941_S_4420', '941_S_4420']


CHECK WHICH SUBJECT ARE AVAILABLE IN OS DIR BUT NOT DATAFRAME

In [15]:
for item in os_id_subject_list:
    if item not in df_id_subject_list:
        print(item)

In [16]:
print("037_V_4001" in os_id_subject_list)
print("037_S_4001" in os_id_subject_list)

print("037_V_4001" in df_id_subject_list)
print("037_S_4001" in df_id_subject_list)

False
True
False
True


In [17]:
for ii in range(len(os_id_subject_list)):
    if os_id_subject_list[ii] == "037_V_4001":
        print("Position of '037_V_4001' in os_id_subject_list",ii)
        
    if os_id_subject_list[ii] == "037_S_4001":
        print("Position of '037_S_4001' in os_id_subject_list",ii)

Position of '037_S_4001' in os_id_subject_list 1067
Position of '037_S_4001' in os_id_subject_list 1068
Position of '037_S_4001' in os_id_subject_list 1069


In [18]:
for ii in range(len(df_id_subject_list)):
    if df_id_subject_list[ii] == "037_V_4001":
        print("Position of '037_V_4001' in os_id_subject_list",ii)
        
    if df_id_subject_list[ii] == "037_S_4001":
        print("Position of '037_S_4001' in os_id_subject_list",ii)

Position of '037_S_4001' in os_id_subject_list 1174
Position of '037_S_4001' in os_id_subject_list 1175
Position of '037_S_4001' in os_id_subject_list 1176


In [19]:
# print(df_id_subject_list)

CHECK WHICH SUBJECT ARE AVAILABLE IN BUT NOT OS DIR

In [20]:
for item in df_id_subject_list:
    if item not in os_id_subject_list:
        print(item)

GET ALL IMAGE ID IN GET OS DIR

In [21]:
os_id_image_list = []

for dirname, _, filenames in os.walk(root_train_dec):
    for filename in filenames:                 
        # print("FILENAME:",filename)
            
        id_image= filename[filename.index('_I')+1 : filename.index('.nii')]
        # print("ID_IMAGE:",id_image)        
                    
        # id_subject = filename[5:15]
        # print("ID_SUBJECT:",id_subject)
            
        # row_index = df.index[df['Image Data ID'] == id_image].tolist()
        # label = df['Group'][row_index[0]]
        # print("LABEL:",label)
        
        os_id_image_list.append(id_image)
        
print(os_id_image_list[:5])       
print(len(os_id_image_list))   

['I13722', 'I238627', 'I14437', 'I240812', 'I15948']
2090


In [22]:
import numpy as np
len(np.unique(os_id_image_list))

2090

In [23]:
check_duplicate = set()
for item in os_id_image_list:
    if item not in check_duplicate:
        check_duplicate.add(item)
    else:
        print(item)

GET ALL IMAGE ID IN DATAFRAME

In [24]:
df_id_image_list = []
df_id_image_list = df["Image Data ID"].to_list()
      
print(len(df_id_image_list))  
print(df_id_image_list[:5]) 

2090
['I325119', 'I307653', 'I294080', 'I313224', 'I352818']


In [25]:
import numpy as np
len(np.unique(df_id_image_list))

2090

CHECK WHICH IMAGE ARE AVAILABLE IN OS DIR BUT NOT DATAFRAME

In [26]:
for item in os_id_image_list:
    if item not in df_id_image_list:
        print(item)

GET FREQUENCY DICT AND COMPARE ON BOTH OS DIR AND DATA FRAME

In [33]:
df['Image Data ID'].value_counts().to_dict()
df_unique_subject_dict = df['Subject'].value_counts().to_dict()

In [41]:
len(list(df_unique_subject_dict.keys()))
# list(df_unique_subject_dict.items())
# list(df_unique_subject_dict.values())

1004

In [47]:
os_id_subject_list = []
os_id_subject_set = set()

for dirname, _, filenames in os.walk(root_train_dec):
    for filename in filenames:                 
        # print("FILENAME:",filename)
            
        # id_image= filename[filename.index('_I')+1 : filename.index('.nii')]
        # print("ID_IMAGE:",id_image)        
                    
        id_subject = filename[5:15]
        # print("ID_SUBJECT:",id_subject)
            
        # row_index = df.index[df['Image Data ID'] == id_image].tolist()
        # label = df['Group'][row_index[0]]
        # print("LABEL:",label)
        
        os_id_subject_list.append(id_subject)
        os_id_subject_set.add(id_subject)
        
print(os_id_subject_list[:5])       
print(len(os_id_subject_list))  
print(len(os_id_subject_set))

['002_S_0295', '002_S_0295', '002_S_0413', '002_S_0413', '002_S_0559']
2090
1004


In [51]:
# initializing the dict
os_id_subject_dict = {}

# iterating over the list
for item in os_id_subject_list:
   # checking the element in dictionary
   if item in os_id_subject_dict:
      # incrementing the counr
      os_id_subject_dict[item] += 1
   else:
      # initializing the count
      os_id_subject_dict[item] = 1

# printing the frequency
# print(os_id_subject_dict)
# print(os_id_subject_dict.keys())

In [54]:
print(df_unique_subject_dict == os_id_subject_dict)

True


In [59]:
print(len(df_id_subject_list) == len(os_id_subject_list))
# print(df_id_subject_list == os_id_subject_list)

df_id_subject_list.sort()
os_id_subject_list.sort()
print(df_id_subject_list == os_id_subject_list)

cnt = 0
for ii in range(len(df_id_subject_list)):
    if (df_id_subject_list[ii] != os_id_subject_list[ii]):
        print(df_id_subject_list[ii])
        cnt+=1
        

True
True


In [58]:
print(cnt)

0
